Import Libraries and Load Data

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the data with a semicolon as the delimiter
data = pd.read_csv('combined_data.csv', delimiter=';')  

# Display the first few rows to verify the data has been loaded correctly
print(data.head())


  Thumb Curl    Thumb Direction Index Curl   Index Direction Middle Curl  \
0  Half Curl  Diagonal Up Right  Full Curl  Diagonal Up Left   Full Curl   
1  Half Curl  Diagonal Up Right  Full Curl  Diagonal Up Left   Full Curl   
2  Half Curl  Diagonal Up Right  Full Curl  Diagonal Up Left   Full Curl   
3  Half Curl  Diagonal Up Right  Full Curl  Diagonal Up Left   Full Curl   
4  Half Curl  Diagonal Up Right  Full Curl  Diagonal Up Left   Full Curl   

  Middle Direction  Ring Curl Ring Direction Pinky Curl Pinky Direction Label  
0      Vertical Up  Full Curl    Vertical Up    No Curl     Vertical Up     I  
1      Vertical Up  Full Curl    Vertical Up    No Curl     Vertical Up     I  
2      Vertical Up  Full Curl    Vertical Up    No Curl     Vertical Up     I  
3      Vertical Up  Full Curl    Vertical Up    No Curl     Vertical Up     I  
4      Vertical Up  Full Curl    Vertical Up    No Curl     Vertical Up     I  


Encode Categorical Features

In [8]:
# Initialize LabelEncoder
le = LabelEncoder()

# Encode each categorical column
for column in data.columns[:-1]:  # Exclude the 'Label' column
    data[column] = le.fit_transform(data[column].astype(str))  # Ensure the column is string before encoding

# Encode the labels as well
data['Label'] = le.fit_transform(data['Label'].astype(str))

# Display the first few rows to verify the encoding
print(data.head())

   Thumb Curl  Thumb Direction  Index Curl  Index Direction  Middle Curl  \
0           0                2           0                0            0   
1           0                2           0                0            0   
2           0                2           0                0            0   
3           0                2           0                0            0   
4           0                2           0                0            0   

   Middle Direction  Ring Curl  Ring Direction  Pinky Curl  Pinky Direction  \
0                 4          0               4           2                4   
1                 4          0               4           2                4   
2                 4          0               4           2                4   
3                 4          0               4           2                4   
4                 4          0               4           2                4   

   Label  
0      8  
1      8  
2      8  
3      8  
4      8  


Split Data into Training and Testing Sets

In [9]:
# Split the data into features (X) and target (y)
X = data.iloc[:, :-1]  # All columns except the last one
y = data['Label']      # The last column is the target

# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shape of the datasets
print(f'Training data shape: {X_train.shape}')
print(f'Testing data shape: {X_test.shape}')


Training data shape: (3998, 10)
Testing data shape: (1000, 10)


Train the Random Forest Classifier

In [10]:
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test data
y_pred = rf_model.predict(X_test)

# Display the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Print the classification report for more detailed metrics
print(classification_report(y_test, y_pred, target_names=le.inverse_transform(range(len(le.classes_)))))


Accuracy: 0.94
              precision    recall  f1-score   support

           A       0.98      0.93      0.95        56
           B       1.00      1.00      1.00         5
           C       0.87      1.00      0.93        27
           D       1.00      1.00      1.00        36
           E       1.00      1.00      1.00        50
           F       1.00      1.00      1.00        28
           G       0.97      1.00      0.98        31
           H       1.00      1.00      1.00        39
           I       1.00      1.00      1.00        49
           K       0.90      0.98      0.94        58
           L       1.00      1.00      1.00        36
           M       0.71      1.00      0.83        29
           N       0.98      0.79      0.88        71
           O       1.00      0.92      0.96        48
           P       1.00      1.00      1.00        31
           Q       1.00      0.98      0.99        59
           R       0.68      0.85      0.76        60
           S

Saving the model

In [11]:
import joblib

# Save the model to a file
joblib.dump(rf_model, 'asl_rf_model.pkl')

print("Model saved to 'asl_rf_model.pkl'")


import joblib
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Load the trained model
rf_model = joblib.load('asl_rf_model.pkl')

# Define initial types
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# Convert the model to ONNX format
onnx_model = convert_sklearn(rf_model, initial_types=initial_type)

# Save the model
with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


Model saved to 'asl_rf_model.pkl'


In [30]:
import joblib
import numpy as np

# Load the model
model = joblib.load('asl_rf_model.pkl')

# Define the input
input_data = np.array([0, 4, 1, 2, 0, 2, 0, 2, 0, 2]).reshape(1, -1)
second_data = np.array([0, 2, 2, 0, 2, 0, 2, 0, 2, 0]).reshape(1, -1)

# Predict the label
predicted_label = model.predict(second_data)

# Print the predicted label
print(predicted_label)

# Decode the label back to the corresponding ASL letter
asl_letter = le.inverse_transform([predicted_label])


print(f"The predicted ASL letter is: {asl_letter[0]}")




[21]
The predicted ASL letter is: X


/Users/berkebozaci/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/berkebozaci/.pyenv/versions/3.11.3/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [33]:
print(f"The predicted ASL letter is: {le.inverse_transform([0])}, with label 0")
print(f"The predicted ASL letter is: {le.inverse_transform([1])} with label 1")
print(f"The predicted ASL letter is: {le.inverse_transform([2])} with label 2")
print(f"The predicted ASL letter is: {le.inverse_transform([3])} with label 3")
print(f"The predicted ASL letter is: {le.inverse_transform([4])} with label 4")
print(f"The predicted ASL letter is: {le.inverse_transform([5])} with label 5")
print(f"The predicted ASL letter is: {le.inverse_transform([6])} with label 6")
print(f"The predicted ASL letter is: {le.inverse_transform([7])} with label 7")
print(f"The predicted ASL letter is: {le.inverse_transform([8])} with label 8")
print(f"The predicted ASL letter is: {le.inverse_transform([9])} with label 9")
print(f"The predicted ASL letter is: {le.inverse_transform([10])} with label 10")
print(f"The predicted ASL letter is: {le.inverse_transform([11])} with label 11")
print(f"The predicted ASL letter is: {le.inverse_transform([12])} with label 12")
print(f"The predicted ASL letter is: {le.inverse_transform([13])} with label 13")
print(f"The predicted ASL letter is: {le.inverse_transform([14])} with label 14")
print(f"The predicted ASL letter is: {le.inverse_transform([15])} with label 15")
print(f"The predicted ASL letter is: {le.inverse_transform([16])} with label 16")
print(f"The predicted ASL letter is: {le.inverse_transform([17])} with label 17")
print(f"The predicted ASL letter is: {le.inverse_transform([18])} with label 18")
print(f"The predicted ASL letter is: {le.inverse_transform([19])} with label 19")
print(f"The predicted ASL letter is: {le.inverse_transform([20])} with label 20")
print(f"The predicted ASL letter is: {le.inverse_transform([21])} with label 21")
print(f"The predicted ASL letter is: {le.inverse_transform([22])} with label 22")
print(f"The predicted ASL letter is: {le.inverse_transform([23])} with label 23")




The predicted ASL letter is: ['A'], with label 0
The predicted ASL letter is: ['B'] with label 1
The predicted ASL letter is: ['C'] with label 2
The predicted ASL letter is: ['D'] with label 3
The predicted ASL letter is: ['E'] with label 4
The predicted ASL letter is: ['F'] with label 5
The predicted ASL letter is: ['G'] with label 6
The predicted ASL letter is: ['H'] with label 7
The predicted ASL letter is: ['I'] with label 8
The predicted ASL letter is: ['K'] with label 9
The predicted ASL letter is: ['L'] with label 10
The predicted ASL letter is: ['M'] with label 11
The predicted ASL letter is: ['N'] with label 12
The predicted ASL letter is: ['O'] with label 13
The predicted ASL letter is: ['P'] with label 14
The predicted ASL letter is: ['Q'] with label 15
The predicted ASL letter is: ['R'] with label 16
The predicted ASL letter is: ['S'] with label 17
The predicted ASL letter is: ['T'] with label 18
The predicted ASL letter is: ['U'] with label 19
The predicted ASL letter is: 

In [34]:
(f"The predicted ASL letter is: {le.inverse_transform([8])}")

"The predicted ASL letter is: ['I']"